In [12]:
from pyardrone import ARDrone, at
from contextlib import suppress
from threading import Timer
import math
import time
import numpy as np

from zmq import Message


In [20]:
drone = ARDrone(host="192.168.1.1")


In [ ]:
drone.send(at.CONFIG('general:navdata_demo', True))

In [ ]:

drone.navdata

In [13]:
from pyardrone import ARDrone, at
from time import sleep, perf_counter
import numpy as np

drone = None
battery = None
vx = None
vy = None
vz = None
yaw = None
pitch = None
roll = None
altitude = None
flying_state = None
navdemo = None
camera_condition = "2"
sending_state = False
current_time = 0

setup_moving = False
running_program = True
moving_program = False
direction = 0
kec = 0.2

id = 0
ip = '192.168.1.1'
camera_ip = 0


# Fungsi untuk menyimpan Navdata ke dalam DataFrame
navdata_records = [['current_time , battery , flying_state , altitude , vx , vy , vz , yaw , pitch , roll']]

def connect():
    global drone
    global ip
    if drone is None:
        try:
            drone = ARDrone(host=ip)
            drone.send(at.CONFIG('general:navdata_demo', True))
        except:
            print("can not connected")
            drone = None
    else:
        if drone.state.navdata_bootstrap is not True:
            navdemo = drone.navdata.demo
            print("connected")
        else:
            print("navdata demo is problem")
            drone.send(at.CONFIG('general:navdata_demo', True))
            sleep(1.5)
            navdemo = drone.navdata.demo
    sleep(0.5)

def disconnect():
    drone.close()

def get_battery():
    global battery
    if drone is not None:
        if drone.navdata_ready.is_set():
            if navdemo is not None:
                battery = navdemo.vbat_flying_percentage
                print(battery)
    else:
        battery = "Not connected"
    return battery

def get_navdata():
    global navdemo
    global current_time
    if sending_state is False:
        if drone is not None:
            if navdemo is not None:
                if drone.state.navdata_bootstrap is not True:
                    navdemo = drone.navdata.demo
                    vx = navdemo.vx/1000
                    vy = navdemo.vy/1000
                    vz = navdemo.vz/1000
                    yaw = navdemo.psi/1000
                    pitch = navdemo.theta/1000
                    roll = navdemo.phi/1000
                    altitude = navdemo.altitude/1000
                    flying_state = drone.state.fly_mask
                    battery = navdemo.vbat_flying_percentage
                    navdata_records.append([current_time , battery , flying_state , altitude , vx , vy , vz , yaw , pitch , roll])
            else:
                if drone.state.navdata_bootstrap is not True:
                    navdemo = drone.navdata.demo
                else:
                    print("navdata demo is problem")
                    drone.send(at.CONFIG('general:navdata_demo', True))

def move(forward=0.0, backward=0.0, right=0.0, left=0.0, up=0.0, down=0.0, cw=0.0, ccw=0.0, dir_speed=0.0):
    if drone != None:
        drone.move(forward=forward, backward=backward,right=right, left=left, up=up, down=down, cw=cw, ccw=ccw)

def hover():
    if drone != None:
        drone.move(forward=0.0, backward=0.0, right=0.0, left=0.0, up=0.0, down=0.0, cw=0.0, ccw=0.0)
        print('drone berhasil hover')
        sleep(1)

def takeoff():
    print('takeoff')
    drone.takeoff()

def land():
    print('landing')
    drone.land()

def read_config():
    pass

def trim():
    drone.send(at.FTRIM())
    sleep(2)
    # drone.send(
    #     at.CONFIG('video:bitrate_control_mode', '1'))
    # sleep(1)
    # drone.send(
    #     at.CONFIG('video:video_fps', '30'))
    # sleep(1)
    # drone.send(
    #     at.CONFIG('video:video_codec', '129'))
    # sleep(1)
    # drone.send(
    #     at.CONFIG('video:max_bitrate', '4000'))

def calib():
    drone.send(at.CALIB(0))

def change_camera2():
    sending_state = True
    camera_condition = "2"
    # sleep(0.5)
    drone.send(at.CONFIG("video:video_channel", camera_condition))
    sending_state = False
    pass

def change_camera3():
    sending_state = True
    camera_condition = "3"
    # sleep(0.5)
    drone.send(at.CONFIG("video:video_channel", camera_condition))
    sending_state = False
    pass

In [2]:
waktu_input = 0.1
waktu_move = 3

# connect()
import time
from pyardrone import ARDrone
drone = ARDrone()
drone.navdata_ready.wait()  # wait until NavData is ready

try : 
    if drone != None:
    
        init_time = perf_counter()
        last_time_sampling = init_time
        last_time_move = init_time

        drone.navdata_ready.wait()  # wait until NavData is ready
        # trim()
        # print("Drone berhasil trim")

        while running_program == True:
        
            current_time = perf_counter()
            get_navdata()
          
            if setup_moving == False:    
                takeoff()
                sleep(5)  # Tunggu beberapa detik untuk memastikan drone stabil
                print("Drone Take off")
                # calib()
                # time.sleep(5)
                # print("Drone berhasil Kalibrasi")
                setup_moving = True

            if (((current_time - last_time_move) >= waktu_move) and (setup_moving == True)) :
                
                if (direction == 0):
                    direction = 1
                    print("Drone Maju")

                elif (direction == 1):
                    hover()
                    direction = 2
                    print("Drone Ke kanan")

                elif (direction == 2):
                    hover()
                    direction = 3
                    print("Drone ke belakang")
                
                elif (direction == 3):
                    hover()
                    direction = 4
                    print("Drone ke kiri")
                
                elif (direction == 4):
                    hover()
                    direction = 5
                    print("Trajektori selesai, akan mendarat")
                
                last_time_move = current_time

            if setup_moving == True :

                if (direction == 1):
                    move(forward = kec)

                elif (direction == 2):
                    move(right = 0.1)
                
                elif (direction == 3):
                    move(backward = 0.1)
                
                elif (direction == 4):
                    move(left = kec)
                
                elif (direction == 5):
                    land()
                    print("Drone Mendarat")
                    running_program == False
                    break

except KeyboardInterrupt:
    print("Program dihentikan oleh pengguna. Drone akan Mendarat")
    land()
    print("Drone mendarat")
    running_program == False

np.savetxt("testDataArray.txt", navdata_records, fmt="%s")
disconnect()

takeoff
Drone Take off
Drone Maju
drone berhasil hover
Drone Ke kanan
drone berhasil hover
Drone ke belakang
drone berhasil hover
Drone ke kiri
drone berhasil hover
Trajektori selesai, akan mendarat
landing
Drone Mendarat


c:\Python310\lib\site-packages\numpy\lib\npyio.py:1513: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X = np.asarray(X)


In [7]:
import time
from pyardrone import ARDrone
drone = ARDrone()
drone.navdata_ready.wait()  # wait until NavData is ready
drone.takeoff()


In [16]:
connect()

connected


In [176]:
for i in range(1000):
    get_navdata()
np.savetxt("testDataArray.txt", navdata_records, fmt="%s")

In [4]:
trim()

In [6]:
takeoff()

takeoff


In [7]:
calib()

In [8]:
land()

landing


In [18]:
disconnect()

In [42]:
print(init_time)
print(current_time)
print(last_time_move)
print(last_time_sampling)

13624.438714
13649.4999339
13624.438714
13649.4999339


In [17]:
change_camera3()

Aruco.py

In [ ]:
import cv2


active_camera = False
active_detector = False
ids = None
draw = False
brightness = 1
contrast = 0.75
ret = False
frame = None
image_scale = 1
id_0 = []

# Inisialisasi dictionary ArUco dan parameter detektor
aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_1000)
aruco_params = cv2.aruco.DetectorParameters()

def connect_camera():
    global ip
    url = ip
    camera_url = 'tcp://'+url+':5555' if url != 0 else 0
    camera = cv2.VideoCapture(camera_url)
    # Periksa apakah kamera berhasil dibuka
    if not camera.isOpened():
        active_camera = False
        print("Error: Tidak dapat membuka kamera.")
    else :
        active_camera = True

def read():
    # Baca frame dari kamera
    ret, frame = camera.read()

    # Periksa apakah frame berhasil dibaca
    if ret == False:
        print("Error: Tidak dapat membaca frame dari kamera.")

def findArucoMarkers():
    # Konversi frame ke grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Deteksi marker pada frame
    corners, ids, rejected = cv2.aruco.detectMarkers(gray, aruco_dict, parameters=aruco_params)
    
    # Gambar deteksi marker pada frame
    if ids is not None:
        cv2.aruco.drawDetectedMarkers(frame, corners, ids)

    elif ids

def get_ids():
    if (len(id_0) != len(ids)) or ((len(id_0) == len(ids)) and (id_0[0] != ids[0])):
        print("ArUco markers detected. IDs: ", ids)
        id_0 = ids
        return ids
    pass

def show():
    # Tampilkan frame dengan deteksi marker
    cv2.imshow('Detected ArUco Markers', frame)

def done():
    camera.release()
    cv2.destroyAllWindows()
    print("done")

def run():
    read()
    if active_camera == True:

        if ret:
            frame = cv2.cvtColor(raw_frame, cv2.COLOR_BGR2RGB)

            if active_detector == True:
                if draw:
                    cv2.aruco.drawDetectedMarkers(frame, corners, ids)

    if active_detector == True:
        if ret:
            findArucoMarkers()

Program pemrosesan aruco

In [1]:
#--------------------------------------------------------------#
############################ IMPORT ############################
#--------------------------------------------------------------#

import cv2
import numpy as np
import os
from time import sleep, perf_counter
import matplotlib
matplotlib.use('TkAgg')  # Pastikan menggunakan backend GUI yang tepat
import matplotlib.pyplot as plt

#--------------------------------------------------------------#
####################### DEFINISI FUNCTION ######################
#--------------------------------------------------------------#

def movAvg(window,dataMovAvgX,dataMovAvgY,xEst,yEst):
    if len(dataMovAvgX) >= (window-1):
        dataMovAvgX.append(xEst)
        dataMovAvgY.append(yEst)
        xAvg = (sum(dataMovAvgX)/len(dataMovAvgX))
        yAvg = (sum(dataMovAvgY)/len(dataMovAvgY))
        dataMovAvgX.pop(0)
        dataMovAvgY.pop(0)
    elif len(dataMovAvgX) < (window-1):
        dataMovAvgX.append(xEst)
        dataMovAvgY.append(yEst)
        xAvg = xEst
        yAvg = yEst
    return xAvg, yAvg,dataMovAvgX,dataMovAvgY

def saveImage(output_folder,file_counter,frame):
    # Simpan gambar ke file
    filename = os.path.join(output_folder, f'detected_marker_{file_counter:04d}.png')
    cv2.imwrite(filename, frame)
    file_counter += 1
    return file_counter
    # kalo mau pake fungsi, copas aja gini -> file_counter = saveImage(output_folder,file_counter,frame)

def dataAruco(num_rows,num_cols,jarak):
        
    total_markers = num_rows * num_cols
    databasePos = []
    dataID = []
    i = 1
    for row in range(num_cols):
        for col in range(num_rows):
            posX = jarak*row
            posY = jarak*col
            databasePos.append([posX, posY])
            dataID.append(i)
            i += 1
    # print(np.array(databasePos))
    # print(databasePos[dataID.index(500)])
    return dataID, databasePos

def convertPos(image, ids, corners, dataID, databasePos):

    ###### est posisi berdasarkan database posisi real #####
    sumx = 0
    sumy = 0
    for i in range(len(ids)): 
        sumx += databasePos[dataID.index(ids[i])][0]
        sumy += databasePos[dataID.index(ids[i])][1]

    x = sumx/len(ids)
    y = sumy/len(ids)

    ###### titik tengah setiap marker aruco #####
    tengah = []
    
    for i in range(len(ids)):
        xTengah = (int(corners[i][0][0][0])+int(corners[i][0][1][0])+int(corners[i][0][2][0])+int(corners[i][0][3][0]))/4
        yTengah = (int(corners[i][0][0][1])+int(corners[i][0][1][1])+int(corners[i][0][2][1])+int(corners[i][0][3][1]))/4
        tengah.append([xTengah, yTengah])

    ###### titik tengah rata rata (est posisi dalam pixel) #####
    sumtengahx = 0
    sumtengahy = 0
    for i in range(len(ids)):
        sumtengahx += tengah[i][0]
        sumtengahy += tengah[i][1]

    posPixelX = int(sumtengahx/len(ids))
    posPixelY = int(sumtengahy/len(ids))

    ###### Pusat Kamera #####
    height, width = image.shape[:2]
    pusatKameraX = int(width/2)
    pusatKameraY = int(height/2)

    ###### est posisi pusat kamera #####
    aX = 0.123
    aY = 0.131
    rXPixel = pusatKameraX - posPixelX
    rYPixel = pusatKameraY - posPixelY
    rX = rXPixel * aX
    rY = rYPixel * aY
    xEst = x + rX
    yEst = y + rY

    return x,y,posPixelX,posPixelY,xEst,yEst,pusatKameraX,pusatKameraY,tengah

def cameraDisplay(frame, corners, ids, posPixelX, posPixelY, pusatKameraX, pusatKameraY, tengah):
    if ids is not None:
        radius = 5
        #Gambar deteksi marker pada frame
        cv2.aruco.drawDetectedMarkers(frame, corners, ids)

        #Gambar titik pusat setiap marker aruco yang terdeteksi
        for i in range(len(ids)):
            xTengah = tengah[i][0]
            yTengah = tengah[i][1]
            cv2.circle(frame,(int(xTengah),int(yTengah)), int(radius) , (0,0,255), -1)
        
        # Gambar titik estimasi MFP
        cv2.circle(frame,(posPixelX,posPixelY), int(2*radius), (0,255,0), -1)

    # Gambar titik Pusat kamera
    cv2.circle(frame,(pusatKameraX,pusatKameraY), int(2*radius), (255,0,0), -1)

    # Tampilkan frame dengan deteksi marker
    cv2.imshow('Detected ArUco Markers', frame)

    return

#--------------------------------------------------------------#
######################### SET UP ###############################
#--------------------------------------------------------------#

id_min1 = []
ids = []
tengah = []
dataAkhir = [["Waktu", "X_movAvg", "y_movAvg", "Pusat x", "Pusat y", "MFP X", "MFP Y", "Pixel pusat X", "Pixel pusat Y", "Pixel MFP X", "Pixel MFP Y"]]
num_rows = 25
num_cols = 20
id_1 = []
window = 1
dataMovAvgX = []
dataMovAvgY = []
plotX = []
plotY = []

# Buat folder baru jika belum ada
output_folder = 'detected_frames'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# membuat database posisi tiap marker aruco
dataID, databasePos = dataAruco(num_rows,num_rows,19.5)

# Inisialisasi counter untuk penamaan file berurutan
file_counter = 0

# Inisialisasi waktu
init_time = perf_counter()
current_time = 0

# Inisialisasi kamera
ardrone_url = 'tcp://192.168.1.1:5555'
cap = cv2.VideoCapture(ardrone_url)

# Periksa apakah kamera berhasil dibuka
if not cap.isOpened():
    print("Error: Tidak dapat membuka kamera.")
    exit()

# Inisialisasi dictionary ArUco dan parameter detektor
aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_1000)
aruco_params = cv2.aruco.DetectorParameters()

plotDisplay = True

if plotDisplay == True:
    # Enable interactive mode
    plt.ion()

    # Create subplot and figure
    figure, ax = plt.subplots(figsize=(10, 8))
    line1, = ax.plot(plotX, plotY, color='blue', marker='.',linewidth=0.5, markersize=15)

    # Set limits for the x and y axis
    ax.set_xlim(0,6)
    ax.set_ylim(0,6)

    # Set labels
    plt.xlabel("X-axis")
    plt.ylabel("Y-axis")
    plt.title("Updating plot...")

#--------------------------------------------------------------#
####################### LOOP UTAMA #############################
#--------------------------------------------------------------#

while True:
    current_time = perf_counter()
    # Baca frame dari kamera
    ret, frame = cap.read()

    # Periksa apakah frame berhasil dibaca
    if not ret:
        print("Error: Tidak dapat membaca frame dari kamera.")
        break

    # Konversi frame ke grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Deteksi marker pada frameco_dict, parameters=aruco_params)
    corners, ids, rejected = cv2.aruco.detectMarkers(gray, aruco_dict, parameters=aruco_params)

    # pemrosesan bila terdeteksi adanya marker aruco
    if ids is not None:

        # Convert bacaan MFP menjadi posisi
        x,y,posPixelX,posPixelY,xEst,yEst,pusatKameraX,pusatKameraY,tengah = convertPos(frame, ids, corners, dataID, databasePos)
        xAvg, yAvg, dataMovAvgX, dataMovAvgY = movAvg(window,dataMovAvgX,dataMovAvgY,xEst,yEst)

        if plotDisplay == True:
            # update data x dan y untuk live plotting data
            plotX.append(xAvg)
            plotY.append(yAvg)

            # Register the value of x and y for live plotting data
            datax = np.array(plotX)
            datay = np.array(plotY)
            line1.set_xdata(datax )
            line1.set_ydata(datay)

        #save data
        dataAkhir.append([current_time, xAvg, yAvg, xEst, yEst, x, y, pusatKameraX, pusatKameraY, posPixelX, posPixelY])
        
        if (len(id_1) != len(ids)) or ((len(id_1) == len(ids)) and (id_1[0] != ids[0])):
            # print("ArUco markers detected. IDs: ", ids)
            id_1 = ids
    
    cameraDisplay(frame, corners, ids, posPixelX, posPixelY, pusatKameraX, pusatKameraY, tengah)

    if plotDisplay == True:
        # Re-draw the figure
        ax.relim()
        ax.autoscale_view()
        figure.canvas.draw()
    
        # To flush the GUI events
        figure.canvas.flush_events()
        plt.pause(0.1)
    
    # Tekan 'q' untuk keluar dari loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

#--------------------------------------------------------------#
######################### ENDING ###############################
#--------------------------------------------------------------#

# Lepaskan kamera dan tutup jendela
cap.release()
cv2.destroyAllWindows()
np.savetxt("Data_Akhir.txt", dataAkhir, fmt="%s")

if plotDisplay == True:
    # Turn off interactive mode
    plt.ioff()

    # Keep the plot window open
    plt.show(block=True)

Error: Tidak dapat membuka kamera.
Error: Tidak dapat membaca frame dari kamera.


: 